# RL Custom environment - Optimal Shower temperature seting

### Youtube reference:https://www.youtube.com/watch?v=Mut_u40Sqz4

# 1. Import Dependencies

In [3]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Types of Spaces

In [3]:
Discrete(3)

Discrete(3)

In [4]:
Box(0,1,shape=(3,3)).sample()

array([[0.78531146, 0.90565854, 0.5292398 ],
       [0.7979142 , 0.29862782, 0.84011114],
       [0.3566075 , 0.9896476 , 0.8726066 ]], dtype=float32)

In [5]:
Box(0,255,shape=(3,3), dtype=int).sample()

array([[ 15, 168, 216],
       [ 89,  11, 165],
       [159, 128, 254]])

In [6]:
Tuple((Discrete(2), Box(0,100, shape=(1,)))).sample()

(1, array([56.831707], dtype=float32))

In [7]:
Dict({'height':Discrete(2), "speed":Box(0,100, shape=(1,))}).sample()

OrderedDict([('height', 1), ('speed', array([34.22735], dtype=float32))])

In [8]:
MultiBinary(4).sample()

array([1, 1, 1, 1], dtype=int8)

In [9]:
MultiDiscrete([5,2,2]).sample()

array([3, 0, 1], dtype=int64)

# 3. Building an Environment

- build agent to give us best shower temperture
- random temperature
- best temp 37 to 39

In [20]:
Box(low=np.array([0]), high=np.array([100])).sample()

F:\anaconda\lib\site-packages\gym\logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


array([56.858707], dtype=float32)

In [21]:
Box(low=0, high=100, shape=(1,)).sample()

array([57.461178], dtype=float32)

In [4]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        # Reset shower time
        self.shower_length = 60 
        return self.state

In [5]:
env=ShowerEnv()

F:\anaconda\lib\site-packages\gym\logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


In [6]:
env.observation_space.sample()

array([5.4218526], dtype=float32)

In [7]:
env.reset()

array([38.])

In [8]:
from stable_baselines3.common.env_checker import check_env

In [9]:
check_env(env)

# 4. Test Environment

In [10]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-60
Episode:2 Score:-10
Episode:3 Score:-58
Episode:4 Score:-22
Episode:5 Score:-44


In [11]:
env.close()

# 5. Train Model

In [12]:
log_path = os.path.join('Training', 'Logs')

In [13]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [17]:
model.learn(total_timesteps=40000)

Logging to Training\Logs\PPO_2
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -9.94    |
| time/              |          |
|    fps             | 569      |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -15         |
| time/                   |             |
|    fps                  | 239         |
|    iterations           | 2           |
|    time_elapsed         | 17          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.006381296 |
|    clip_fraction        | 0.0319      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.73       |
|    explained_variance   | 0.00873     |

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -6.34       |
| time/                   |             |
|    fps                  | 141         |
|    iterations           | 11          |
|    time_elapsed         | 159         |
|    total_timesteps      | 22528       |
| train/                  |             |
|    approx_kl            | 0.000996329 |
|    clip_fraction        | 0.0282      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.507      |
|    explained_variance   | -0.00353    |
|    learning_rate        | 0.0003      |
|    loss                 | 56.6        |
|    n_updates            | 280         |
|    policy_gradient_loss | -0.0012     |
|    value_loss           | 105         |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60    

# 6. Save Model

In [18]:
model.save('PPO')

In [19]:
evaluate_policy(model, env, n_eval_episodes=10, render=False)

(-36.0, 48.0)